In [4]:
import sys
!{sys.executable} -m pip install pip earthengine-api
!{sys.executable} -m pip install pip geemap

  Using cached earthengine_api-0.1.328-py3-none-any.whl
  Using cached httplib2-0.20.4-py3-none-any.whl (96 kB)
  Using cached httplib2shim-0.0.3-py2.py3-none-any.whl
  Using cached google_api_python_client-1.12.11-py2.py3-none-any.whl (62 kB)
  Using cached google_auth_httplib2-0.1.0-py2.py3-none-any.whl (9.3 kB)
  Using cached uritemplate-3.0.1-py2.py3-none-any.whl (15 kB)
  Using cached geemap-0.17.1-py2.py3-none-any.whl (2.1 MB)
  Using cached ee_extra-0.0.14-py3-none-any.whl
  Using cached geeadd-0.5.6-py3-none-any.whl (30 kB)
  Using cached sankee-0.2.0-py3-none-any.whl
  Using cached geocoder-1.38.1-py2.py3-none-any.whl (98 kB)
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  Using cached logzero-1.7.0-py2.py3-none-any.whl (16 kB)
  Using cached plotly-5.10.0-py2.py3-none-any.whl (15.2 MB)
  Using cached ratelim-0.1.6-py2.py3-none-any.whl (4.0 kB)


In [6]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
#from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3
import ee
from shapely.ops import transform, unary_union

In [7]:
service_account = 'climate-hazard-demo@data-portal-adaptation.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'google_cred.json')

In [8]:
import ee
ee.Initialize(credentials)

In [9]:
OUTPUT_FILENAME = 'GRE-2.3.csv'

In [10]:
THRESHOLD = 5.0

In [11]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data"

In [12]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/geo_ref.csv')

# remove cities without tree cover data availability
#tml_not_available_cities = ['BRA-Salvador','MEX-Monterrey']
tml_not_available_cities = []
boundary_georef = boundary_georef[~boundary_georef['geo_name'].isin(tml_not_available_cities)].reset_index(drop=True)
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM2,COD,ADM2
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


In [13]:
pm25_2020 = ee.Image("projects/wri-datalab/Cities4Forests/acag_pm25-surface")
worldpop_2020 = ee.ImageCollection("WorldPop/GP/100m/pop").filter(ee.Filter.eq('year', 2020)).mosaic()

In [14]:
exceedance_2020 = pm25_2020.gte(THRESHOLD)

In [15]:
threshold_percentage = pm25_2020.multiply(1.0/THRESHOLD).multiply(100.0)

In [16]:
exceedance_pop_img = worldpop_2020.mask(exceedance_2020)

In [25]:
def get_exceedancepops(fc):
    totalpop = worldpop_2020.reduceRegions(**{
        'reducer': ee.Reducer.sum(),
        'collection': fc,
        'scale': 10
    })
    exceedancepop = exceedance_pop_img.reduceRegions(**{
        'reducer': ee.Reducer.sum(),
        'collection': fc,
        'scale': 10
    })
    tmp = geemap.ee_to_pandas(exceedancepop)['sum'] / geemap.ee_to_pandas(totalpop)['sum']
    iszero = tmp == 0
    res = tmp + (iszero * -9999)
    return res

def get_meanconc(fc):
    meanconc = threshold_percentage.reduceRegions(**{
        'reducer': ee.Reducer.mean(),
        'collection': fc,
        'scale': 10
    })
    tmp = geemap.ee_to_pandas(meanconc)['mean']
    iszero = tmp == 0
    res = tmp + (iszero * -9999)
    return res

#    return threshold_percentage.reduceRegion(ee.Reducer.mean(), geom, 100, 'epsg:4326').getInfo()['b1']

In [65]:
results = []
for i in range(len(boundary_georef)):
    geo_name = boundary_georef.loc[i, 'geo_name']
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    for boundary_id in [boundary_id_aoi, boundary_id_unit]:
        if boundary_id[-3:] != 'nan':
            print("\n geo_name: " + boundary_id)
            # read boundaries
            boundary_path = aws_s3_dir + '/boundaries/v_0/boundary-'+ boundary_id + '.geojson'
            boundary_geo = requests.get(boundary_path).json()
            temp_gdf = gpd.GeoDataFrame.from_features(boundary_geo)
            
            #temp_gdf['mean_conc_2020'] = temp_gdf.apply(meanconc)
            
            boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
            temp_gdf['exceedance_popfraction_2020'] = get_exceedancepops(boundary_geo_ee)
            temp_gdf['mean_concentration_2020'] = get_meanconc(boundary_geo_ee).fillna(-9999)
            
            #temp_gdf['mean_concentration_2020'] = temp_gdf.apply(lambda x: -9999 if x['tmp_mean_concentration_2020'].isna() else x['tmp_mean_concentration_2020'], axis=1)
            results.append(temp_gdf)
            gre_2_3 = pd.concat(results, axis=0)[['geo_id', 'geo_level', 'geo_name', 'geo_parent_name', 'exceedance_popfraction_2020', 'mean_concentration_2020']]
            gre_2_3.to_csv(OUTPUT_FILENAME)
print('Done')


 geo_name: BRA-Salvador-ADM4union

 geo_name: BRA-Salvador-ADM4

 geo_name: COD-Bukavu-ADM3union

 geo_name: COD-Bukavu-ADM3

 geo_name: COD-Uvira-ADM3union

 geo_name: COD-Uvira-ADM3

 geo_name: COG-Brazzaville-ADM4union

 geo_name: COG-Brazzaville-ADM4

 geo_name: COL-Barranquilla-ADM4union

 geo_name: COL-Barranquilla-ADM4

 geo_name: ETH-Addis_Ababa-ADM4union

 geo_name: ETH-Addis_Ababa-ADM4

 geo_name: ETH-Dire_Dawa-ADM3union

 geo_name: ETH-Dire_Dawa-ADM3

 geo_name: KEN-Nairobi-ADM3union

 geo_name: KEN-Nairobi-ADM3

 geo_name: MDG-Antananarivo-ADM4union

 geo_name: MDG-Antananarivo-ADM4

 geo_name: MEX-Mexico_City-ADM2union

 geo_name: MEX-Mexico_City-ADM2

 geo_name: MEX-Monterrey-ADM2union

 geo_name: MEX-Monterrey-ADM2
Done
